In [1]:
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
from hurst import compute_Hc
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import ADXIndicator
from ta.volatility import AverageTrueRange
from ta.trend import SMAIndicator
from ta.momentum import RSIIndicator
from ta.volume import VolumeWeightedAveragePrice

import yfinance as yf
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
access_token = "9c7349b9a9bd3d17409758cb7e29e53f-7fcbdfe7bc0636788aa51f7e4a95601f"
account_id = "101-003-28600525-001"

accountID = account_id
access_token = access_token

api = API(access_token=access_token)

In [3]:
class Common_Class():

  def __init__(self, instrument, params, capital, transaction_cost, verbose = True):

    self.instrument = instrument
    self.params = params
    self.initial_capital = capital # this is the initial capital you want to trade with
    self.capital = capital # this capital will change depending on trades
    self. transaction_cost = transaction_cost # the transaction cost for trading
    self.quantity = 0 # quantities to buy/sell
    self.position = 0 # the trades in progress, long or short
    self.trades = 0 # Number of trades
    self.verbose = verbose # if you want to see detailed output (logs)

    self.prepare_data() # prepares the data
    self.daily_capital = pd.DataFrame(index = self.data.index, columns=["Capital", "Portfolio Value"])
    self.daily_capital["Capital"].iloc[0] = self.capital
    self.daily_capital["Portfolio Value"].iloc[0] = self.capital

  def prepare_data(self):
    # since we are building a common class for all types of strategy, we will not calcualte the moving averages now.
    # we will calculate the returns though.
    # Since most strategies utilise close prices we are only factoring close price. However, you can alter acoordingly.

    # Make a request
    r = instruments.InstrumentsCandles(instrument=self.instrument, params=self.params)
    data = api.request(r)

    # Extract date and closing price and form as a DataFrame
    dates = [candle['time'][:19] for candle in data['candles']]  # Intercept the first 10 characters to get the date
    close_prices = [float(candle['mid']['c']) for candle in data['candles']]  # Get Close Price

    # Create DataFrame, row_index: date; column: close price
    bt_data = pd.DataFrame(data=close_prices, index=pd.to_datetime(dates), columns=['Close_Price'])
    bt_data['Return'] = np.log(bt_data["Close_Price"] / bt_data["Close_Price"].shift(1))
    bt_data = bt_data.dropna()
    self.data = bt_data

  def update_daily_portfolio(self, bar):
      if pd.isnull(self.daily_capital["Capital"].iloc[bar]):
        self.daily_capital["Capital"].iloc[bar] = self.daily_capital["Capital"].iloc[bar-1]
        self.daily_capital["Portfolio Value"].iloc[bar] = self.daily_capital["Capital"].iloc[bar] + self.quantity * self.data.Close_Price.iloc[bar]

  def close_graph(self):
    plt.figure(figsize=(15, 5))
    plt.plot(self.data["Close_Price"] ,color='black', label='Price', linestyle='dashed')
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.title("Close Prices of {}".format(self.instrument))
    plt.legend()
    plt.grid()
    plt.show()

  def return_date_price(self, bar):

    # A bar is a unit of data at a given time, depends on the interval you choose, it provides you OHLCV and time info
    # Since we have modeled close prices, we will get the price and date

    date = str(self.data.index[bar])[:10] #First 10 contains the date elements, rest is time
    price = self.data.Close_Price.iloc[bar]
    return date, price

  def realised_balance(self, bar):

    #Returns you the realised capital in your account at a given time period / bar

    date, price = self.return_date_price(bar)
    print("Date :{} | Realised Balance: {:0.1f}".format(date,self.capital))

  def unrealised_balance(self, bar):

    #Returns you the unrealised capital (trades in progress) in your account at a given time period / bar

    date, price = self.return_date_price(bar)
    ub = self.quantity *price
    print("Date :{} | Unrealised Balance: {:0.1f}".format(date,ub))

  def total_balance(self, bar):

    #Unrealised plus realised

    date, price = self.return_date_price(bar)
    tb = self.quantity *price + self.capital
   # tb = self.quantity *price + self.capital
    print("Date :{} | Total Balance: {:0.1f}".format(date,tb))

  def buy_order(self,bar,quantity=None, dollar =None ):
    date, price = self.return_date_price(bar)
    if quantity == None:
      quantity = int(dollar/price)
    self.capital = self.capital - ((quantity * price)*(1 + self.transaction_cost)) # capital will be lost in buying
    self.quantity = self.quantity + quantity
    self.trades = self.trades + 1

    self.daily_capital["Capital"].iloc[bar] = self.capital
    self.daily_capital["Portfolio Value"].iloc[bar] = self.capital + self.quantity * price

    if self.verbose:
      print("Bought {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(quantity,self.instrument, price, quantity * price))
      self.realised_balance(bar)
      self.unrealised_balance(bar)
      self.total_balance(bar)


  def sell_order(self,bar,quantity=None, dollar=None ):
    date, price = self.return_date_price(bar)
    if quantity == None:
      quantity = int(dollar/price)
    self.capital = self.capital + ((quantity * price)*(1 - self.transaction_cost)) # capital will be added after selling
    self.quantity = self.quantity - quantity
    self.trades = self.trades + 1

    self.daily_capital["Capital"].iloc[bar] = self.capital
    self.daily_capital["Portfolio Value"].iloc[bar] = self.capital + self.quantity * price
    
    if self.verbose:
      print("Sold {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(quantity,self.instrument, price, quantity * price))
      self.realised_balance(bar)
      self.unrealised_balance(bar)
      self.total_balance(bar)

  def last_trade(self, bar):
    date, price = self.return_date_price(bar)
    last_quantity = self.quantity # this variable to print and store as self.quantity will be set to 0 later
    self.capital = self.capital + self.quantity * price
    self.quantity = 0 # as no more quantity now. all will be settled
    self.trades = self.trades +1
    if self.verbose:
      print("Closed open trades for {} shares of {} at {:0.1f} per share worth {:0.1f} $".format(last_quantity,self.instrument, price, last_quantity * price))
      self.total_balance(bar)

      returns = (self.capital - self.initial_capital) /self.initial_capital *100
      print("The total capital at end of strategy: {:0.1f}".format(self.capital))
      print( "The strategy returns on investment are {:0.1f} %".format(returns))
      print( "Total trades by startegy are {:0.1f}".format(self.trades))

  def calculate_performance_metrics(self):
    # This function will be used to calculate performance metrics of the strategy
    # Calculate the regular returns
    total_log_returns = np.log(self.capital/self.initial_capital)
    total_regular_returns = (np.exp(total_log_returns) - 1)

    # Calculate log daily returns
    daily_log_returns = total_log_returns / (self.data.index[-1] - self.data.index[0]).days
    daily_regular_returns = (np.exp(daily_log_returns) - 1)

    # Calculate the annualized returns
    annual_log_returns = daily_log_returns * 252
    annual_regular_returns = (np.exp(annual_log_returns) - 1)

    # Calculate the sharpe ratio
    sharpe_ratio = annual_regular_returns / (self.daily_capital['Portfolio Value'].pct_change().std() * np.sqrt(252))

    # Calculate the maximum drawdown
    rolling_max = self.daily_capital['Portfolio Value'].cummax()
    daily_drawdown = self.daily_capital['Portfolio Value'] / rolling_max - 1.0
    max_daily_drawdown = daily_drawdown.cummin()

    # Calculate the Calmar ratio
    calmar_ratio = annual_regular_returns / abs(max_daily_drawdown.min())

    # Print the metrics
    print("Total Regular Returns: {:0.1f}%".format(total_regular_returns * 100))
    print("Annual Regular Returns: {:0.1f}%".format(annual_regular_returns * 100))
    print("Sharpe Ratio: {:0.2f}".format(sharpe_ratio))
    print("Max Drawdown: {:0.1f}%".format(abs(max_daily_drawdown.min()) * 100))
    print("Calmar Ratio: {:0.2f}".format(calmar_ratio))


In [4]:
'''
if __name__ == "__main__":
  instrument = "EUR_USD"
  params = {
    "from": "2022-10-07T12:00:00Z",  # 开始时间
    "to": "2022-10-07T15:00:00Z",  # 结束时间
    "granularity": "S5",  # 日线数据
    "price":"M"
    }
  A = Common_Class(instrument, params,10000, 0.0, True)
  A.close_graph()
'''

'\nif __name__ == "__main__":\n  instrument = "EUR_USD"\n  params = {\n    "from": "2022-10-07T12:00:00Z",  # 开始时间\n    "to": "2022-10-07T15:00:00Z",  # 结束时间\n    "granularity": "S5",  # 日线数据\n    "price":"M"\n    }\n  A = Common_Class(instrument, params,10000, 0.0, True)\n  A.close_graph()\n'

In [5]:
class Momentum_Hurst_RSI(Common_Class):

  def go_long(self, bar, quantity = None, dollar = None):
    if self.position == -1:
      self.buy_order(bar, quantity = -self.quantity) #to clear previous short position and therefore negative quantity.
    if quantity:
      self.buy_order (bar, quantity = quantity) # to create new fresh order
    elif dollar:
      if dollar == 'all':
        dollar = self.capital
      self.buy_order(bar, dollar = dollar)


  def go_short(self, bar, quantity = None, dollar = None):
    if self.position == 1:
      self.sell_order(bar, quantity = self.quantity) #to clear previous long vposition
    if quantity:
      self.sell_order (bar, quantity = quantity) # to create new fresh order
    elif dollar:
      if dollar == 'all':
        dollar = self.capital
      self.sell_order(bar, dollar = dollar)

  def run_strategy(self, ST_window, LT_window, hurst_window):
    self.position = 0
    self.trades = 0
    self.capital = self.initial_capital

    # Calculate the momentum and the Hurst exponent
    self.data['ShortTermMomentum'] = self.data['Close_Price'] - self.data['Close_Price'].shift(ST_window)
    self.data['LongTermMomentum'] = self.data['Close_Price'] - self.data['Close_Price'].shift(LT_window)    
    self.data['Hurst'] = self.data['Close_Price'].rolling(window=hurst_window).apply(lambda x: compute_Hc(x)[0])
    # Calculating the RSI
    rsi_indicator = RSIIndicator(close = self.data['Close_Price'], window = ST_window, fillna = False)
    RSI = rsi_indicator.rsi()
    # Adding the RSI to the dataframe
    self.data['RSI'] = RSI


    for bar in range(hurst_window, len(self.data)):
      if self.position in [0,-1]: # checking no position or short position
        if ((self.data['Hurst'].iloc[bar]>0.8) and (self.data['ShortTermMomentum'].iloc[bar]>0) and (self.data['LongTermMomentum'].iloc[bar]>0)) or \
          ((self.data['Hurst'].iloc[bar]<0.2) and (self.data['ShortTermMomentum'].iloc[bar]<0) and (self.data['LongTermMomentum'].iloc[bar]<0)) or \
            ((self.data['Hurst'].iloc[bar]>=0.2 and self.data['Hurst'].iloc[bar]<=0.8) and (self.data['RSI'].iloc[bar]<=30)):
          self.go_long(bar, dollar="all") # go with all money
          self.position = 1 # long created
          print("--------")

      if self.position in [0,1]: # checking no position or long position
        if ((self.data['Hurst'].iloc[bar]>0.8) and (self.data['ShortTermMomentum'].iloc[bar]<0) and (self.data['LongTermMomentum'].iloc[bar]<0)) or \
          ((self.data['Hurst'].iloc[bar]<0.2) and (self.data['ShortTermMomentum'].iloc[bar]>0) and (self.data['LongTermMomentum'].iloc[bar]>0)) or \
            ((self.data['Hurst'].iloc[bar]>=0.2 and self.data['Hurst'].iloc[bar]<=0.8) and (self.data['RSI'].iloc[bar]>70)):
          self.go_short(bar, dollar ="all") # go with all money
          self.position = -1 # short created
          print("--------")

      self.update_daily_portfolio(bar)
      
    print("--------")
    self.last_trade(bar)
    self.calculate_performance_metrics()






In [7]:
if __name__ == "__main__":
  instrument = "EUR_USD"
  params = {
    "from": "2015-10-07T00:00:00Z",  # 开始时间
    "to": "2015-10-07T03:00:00Z",  # 结束时间
    "granularity": "S5",  # 日线数据
    "price":"M"
    }
  A = Momentum_Hurst_RSI(instrument,params,10000,0.01,True)
  A.run_strategy(5,21,200)

Bought 8876 shares of EUR_USD at 1.1 per share worth 9999.3 $
Date :2015-10-07 | Realised Balance: -99.3
Date :2015-10-07 | Unrealised Balance: 9999.3
Date :2015-10-07 | Total Balance: 9900.0
--------
Sold 8876 shares of EUR_USD at 1.1 per share worth 9997.0 $
Date :2015-10-07 | Realised Balance: 9797.7
Date :2015-10-07 | Unrealised Balance: 0.0
Date :2015-10-07 | Total Balance: 9797.7
Sold 8699 shares of EUR_USD at 1.1 per share worth 9797.7 $
Date :2015-10-07 | Realised Balance: 19497.4
Date :2015-10-07 | Unrealised Balance: -9797.7
Date :2015-10-07 | Total Balance: 9699.8
--------
Bought 8699 shares of EUR_USD at 1.1 per share worth 9797.5 $
Date :2015-10-07 | Realised Balance: 9602.0
Date :2015-10-07 | Unrealised Balance: 0.0
Date :2015-10-07 | Total Balance: 9602.0
Bought 8525 shares of EUR_USD at 1.1 per share worth 9601.5 $
Date :2015-10-07 | Realised Balance: -95.6
Date :2015-10-07 | Unrealised Balance: 9601.5
Date :2015-10-07 | Total Balance: 9505.9
--------
Sold 8525 shares o

C:\Users\HYQ\AppData\Local\Temp\ipykernel_20548\2031331083.py:144: RuntimeWarning: divide by zero encountered in scalar divide
  daily_log_returns = total_log_returns / (self.data.index[-1] - self.data.index[0]).days
